## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Namatanai,PG,-3.6667,152.4333,77.40,82,79,6.06,moderate rain
1,1,Norman Wells,CA,65.2820,-126.8329,48.24,71,90,9.22,overcast clouds
2,2,Rockland,US,41.1668,-74.0496,88.86,55,100,1.01,overcast clouds
3,3,Montauban,FR,44.0833,1.5000,69.12,71,33,7.00,scattered clouds
4,4,Svirstroy,RU,60.7999,33.7248,61.25,98,100,9.28,overcast clouds
5,5,Mataura,NZ,-46.1927,168.8643,34.48,99,100,0.94,overcast clouds
6,6,Bethel,US,41.3712,-73.4140,86.88,65,1,9.22,clear sky
7,7,Avarua,CK,-21.2078,-159.7750,77.05,73,5,2.30,clear sky
8,8,Bratslav,UA,48.8226,28.9407,60.01,66,0,5.08,clear sky
9,9,Hobart,AU,-42.8794,147.3294,35.26,88,98,3.00,overcast clouds


In [3]:
# Confirm the data types in the DataFrame
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Namatanai,PG,-3.6667,152.4333,77.40,82,79,6.06,moderate rain
2,2,Rockland,US,41.1668,-74.0496,88.86,55,100,1.01,overcast clouds
6,6,Bethel,US,41.3712,-73.4140,86.88,65,1,9.22,clear sky
7,7,Avarua,CK,-21.2078,-159.7750,77.05,73,5,2.30,clear sky
10,10,Rikitea,PF,-23.1203,-134.9692,74.01,80,11,12.33,few clouds
15,15,Albany,US,42.6001,-73.9662,87.85,64,99,4.03,overcast clouds
16,16,Banda Aceh,ID,5.5577,95.3222,79.63,83,88,7.29,overcast clouds
17,17,Fiumicino,IT,41.7675,12.2291,83.98,75,0,10.36,clear sky
22,22,Flin Flon,CA,54.7682,-101.8650,78.64,61,90,11.50,smoke
23,23,Conceicao Da Barra,BR,-18.5933,-39.7322,70.03,71,90,9.73,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                316
City                   316
Country                316
Lat                    316
Lng                    316
Max Temp               316
Humidity               316
Cloudiness             316
Wind Speed             316
Current Description    316
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No null values found

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Namatanai,PG,77.40,moderate rain,-3.6667,152.4333,
2,Rockland,US,88.86,overcast clouds,41.1668,-74.0496,
6,Bethel,US,86.88,clear sky,41.3712,-73.4140,
7,Avarua,CK,77.05,clear sky,-21.2078,-159.7750,
10,Rikitea,PF,74.01,few clouds,-23.1203,-134.9692,
15,Albany,US,87.85,overcast clouds,42.6001,-73.9662,
16,Banda Aceh,ID,79.63,overcast clouds,5.5577,95.3222,
17,Fiumicino,IT,83.98,clear sky,41.7675,12.2291,
22,Flin Flon,CA,78.64,smoke,54.7682,-101.8650,
23,Conceicao Da Barra,BR,70.03,overcast clouds,-18.5933,-39.7322,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [26]:
hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Namatanai,PG,77.40,moderate rain,-3.6667,152.4333,Seagulls Inn Namatani
2,Rockland,US,88.86,overcast clouds,41.1668,-74.0496,National Camp Association
6,Bethel,US,86.88,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
7,Avarua,CK,77.05,clear sky,-21.2078,-159.7750,Paradise Inn
10,Rikitea,PF,74.01,few clouds,-23.1203,-134.9692,People ThankYou
15,Albany,US,87.85,overcast clouds,42.6001,-73.9662,NaN
16,Banda Aceh,ID,79.63,overcast clouds,5.5577,95.3222,Sulthan Hotel International
17,Fiumicino,IT,83.98,clear sky,41.7675,12.2291,Al Porticciolo Hotel
22,Flin Flon,CA,78.64,smoke,54.7682,-101.8650,Victoria Inn Flin Flon
23,Conceicao Da Barra,BR,70.03,overcast clouds,-18.5933,-39.7322,Cricaré Praia Hotel


In [27]:
# 7. Drop the rows where there is no Hotel Name.
# Solution from classmate Mohamed Metwalli on Slack Aug 2-2021
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [29]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Namatanai,PG,77.40,moderate rain,-3.6667,152.4333,Seagulls Inn Namatani
2,Rockland,US,88.86,overcast clouds,41.1668,-74.0496,National Camp Association
6,Bethel,US,86.88,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
7,Avarua,CK,77.05,clear sky,-21.2078,-159.7750,Paradise Inn
10,Rikitea,PF,74.01,few clouds,-23.1203,-134.9692,People ThankYou
16,Banda Aceh,ID,79.63,overcast clouds,5.5577,95.3222,Sulthan Hotel International
17,Fiumicino,IT,83.98,clear sky,41.7675,12.2291,Al Porticciolo Hotel
22,Flin Flon,CA,78.64,smoke,54.7682,-101.8650,Victoria Inn Flin Flon
23,Conceicao Da Barra,BR,70.03,overcast clouds,-18.5933,-39.7322,Cricaré Praia Hotel
29,Saint-Philippe,RE,71.15,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.3)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

# Add markers for each city on the map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))